In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.3369813859462738
epoch:  1, loss: 0.18830543756484985
epoch:  2, loss: 0.11335527151823044
epoch:  3, loss: 0.075849249958992
epoch:  4, loss: 0.05692458525300026
epoch:  5, loss: 0.04745115339756012
epoch:  6, loss: 0.042780984193086624
epoch:  7, loss: 0.040504369884729385
epoch:  8, loss: 0.03939918056130409
epoch:  9, loss: 0.0388604961335659
epoch:  10, loss: 0.038592249155044556
epoch:  11, loss: 0.03845079988241196
epoch:  12, loss: 0.038372207432985306
epoch:  13, loss: 0.03835640102624893
epoch:  14, loss: 0.03823895752429962
epoch:  15, loss: 0.03817911446094513
epoch:  16, loss: 0.03814564272761345
epoch:  17, loss: 0.038098569959402084
epoch:  18, loss: 0.03803489729762077
epoch:  19, loss: 0.0379982553422451
epoch:  20, loss: 0.03794249892234802
epoch:  21, loss: 0.037877846509218216
epoch:  22, loss: 0.03784297779202461
epoch:  23, loss: 0.037826891988515854
epoch:  24, loss: 0.03776106983423233
epoch:  25, loss: 0.03772670775651932
epoch:  26, loss: 0.

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.745688319128039
Test metrics:  R2 = 0.6817437390731944


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.5688683986663818
epoch:  1, loss: 0.3418840169906616
epoch:  2, loss: 0.22045190632343292
epoch:  3, loss: 0.1485162377357483
epoch:  4, loss: 0.10523427277803421
epoch:  5, loss: 0.07901901006698608
epoch:  6, loss: 0.06309784948825836
epoch:  7, loss: 0.053424522280693054
epoch:  8, loss: 0.04755575582385063
epoch:  9, loss: 0.044009730219841
epoch:  10, loss: 0.041869208216667175
epoch:  11, loss: 0.04057271033525467
epoch:  12, loss: 0.03978190943598747
epoch:  13, loss: 0.03929724171757698
epoch:  14, loss: 0.03899863362312317
epoch:  15, loss: 0.03881295770406723
epoch:  16, loss: 0.03869627043604851
epoch:  17, loss: 0.038677435368299484
epoch:  18, loss: 0.03857744485139847
epoch:  19, loss: 0.038514621555805206
epoch:  20, loss: 0.038497600704431534
epoch:  21, loss: 0.03843778744339943
epoch:  22, loss: 0.03842516243457794
epoch:  23, loss: 0.03836779668927193
epoch:  24, loss: 0.03836705908179283
epoch:  25, loss: 0.03831084817647934
epoch:  26, loss: 0.03

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7134678142158339
Test metrics:  R2 = 0.6677058998053428
